In [1]:
import torch
import nnsight
from PIL import Image
import os, json, argparse, itertools, math, torch, numpy as np
from tqdm import tqdm
from transformers import AutoProcessor
from vllm.inputs import TextPrompt
from nnsight.modeling.vllm import VLLM
import einops
import json

torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [3]:
vllm = VLLM("gpt2", device = "auto", dispatch = True)

INFO 05-04 13:17:09 config.py:2167] Downcasting torch.float32 to torch.float16.
INFO 05-04 13:17:09 config.py:478] This model supports multiple tasks: {'embed', 'reward', 'classify', 'generate', 'score'}. Defaulting to 'generate'.
INFO 05-04 13:17:09 llm_engine.py:249] Initializing an LLM engine (v0.6.5) with config: model='gpt2', speculative_config=None, tokenizer='gpt2', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0,

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 05-04 13:17:11 model_runner.py:1097] Loading model weights took 0.2460 GB
INFO 05-04 13:17:12 worker.py:241] Memory profiling takes 0.47 seconds
INFO 05-04 13:17:12 worker.py:241] the current vLLM instance can use total_gpu_memory (139.72GiB) x gpu_memory_utilization (0.90) = 125.75GiB
INFO 05-04 13:17:12 worker.py:241] model weights take 0.25GiB; non_torch_memory takes -0.02GiB; PyTorch activation peak memory takes 0.46GiB; the rest of the memory reserved for KV Cache is 125.06GiB.
INFO 05-04 13:17:12 gpu_executor.py:76] # GPU blocks: 227659, # CPU blocks: 7281
INFO 05-04 13:17:12 gpu_executor.py:80] Maximum concurrency for 1024 tokens per request: 3557.17x
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_12524/3361498698.py", line 1, in <module>
    vllm = VLLM("gpt2", device = "auto", dispatch = True)
  File "/usr/local/lib/python3.10/dist-packages/nnsight/modeling/vllm/vllm.py", line 68, in __init__
    super().__init__(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/nnsight/modeling/mixins/meta.py", line 22, in __init__
    super().__init__(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/nnsight/modeling/mixins/loadable.py", line 14, in __init__
    model = self._load(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/nnsight/modeling/vllm/vllm.py", line 141, in _load
    llm = LLM(
  File "/usr/local/lib/python3.10/dist-packages/vllm/utils.py", line 990, in inner
  File "/usr/local/lib/python3.10/dist-packages/vllm/entry

In [5]:
with vllm.trace(temperature=0.0, top_p=1.0, max_tokens=1) as tracer:
  with tracer.invoke("The Eiffel Tower is located in the city of"):
    clean_logits = vllm.logits.output.save()

  with tracer.invoke("The Eiffel Tower is located in the city of"):
    vllm.transformer.h[-2].mlp.output[:] = 0
    corrupted_logits = vllm.logits.output.save()



Processed prompts: 100%|██████████| 4/4 [00:00<00:00, 283.31it/s, est. speed input: 3133.64 toks/s, output: 284.72 toks/s]


In [6]:
print("\nCLEAN - The Eiffel Tower is located in the city of", vllm.tokenizer.decode(clean_logits.argmax(dim=-1)))
print("\nCORRUPTED - The Eiffel Tower is located in the city of", vllm.tokenizer.decode(corrupted_logits.argmax(dim=-1)))



CLEAN - The Eiffel Tower is located in the city of  Paris

CORRUPTED - The Eiffel Tower is located in the city of  London
